### GPT API로 50개씩.csv 데이터 돌리기

In [1]:
import pandas as pd
import re
import json
import time

from langchain_openai import ChatOpenAI
import openai
from dotenv import load_dotenv
import os

load_dotenv()

False

In [ ]:
df = pd.read_csv('../50개씩.csv')
df

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=1000,
    openai_api_key="openai_api_key"
)

In [ ]:
example = """
review
'''몇 번째 내 돈 내 산인지 모르겠어요 진짜 쟁여템입니다
이렇게까지 내용물이 풍부한 밀 키트는 처음이에요 간도 딱 맞고 양도 푸짐하고 게다가 양이 진짜 많이 들어서 먹을 때마다 감동입니다.
고기의 품질도 너무 좋아요 보통은 국에 흩어지는 고기들이 많은데 제대로 모양을 갖춘 고기가 한가득 들어 있어서 양 곰탕 집에 와서 먹는 기분이 들어요'''
category_labels
{"진짜 쟁여템입니다":'보관',
"내용물이 풍부한 밀 키트는 처음이에요":'양',
"간도 딱 맞고":'맛/향/풍미',
"양도 푸짐하고":'양',
"양이 진짜 많이 들어서":'양',
"고기의 품질도 너무 좋아요":'맛/향/풍미',
"제대로 모양을 갖춘 고기가 한가득 들어 있어서":'양',
"양 곰탕 집에 와서 먹는 기분이 들어요":'맛/향/풍미'}


review
'''후기 좋아서 사 봤는데 고기가 듬뿍 들었고 팍팍하지 않고 부들부들해요 근데 양 곰탕인데 내장은 몇 개 없어서 아쉬워요
그리고 냉동 제품이지만 제조 일이 오래된 거 아닌가요 재고 세일인 건가라는 생각도 드네요”'''
category_labels
{"고기가 듬뿍 들었고":'양',
"팍팍하지 않고 부들부들해요":'맛/향/풍미',
"내장은 몇 개 없어서 아쉬워요":'양',
"냉동 제품이지만 제조 일이 오래된 거 아닌가요":'포장/배송',
"재고 세일인 건가라는 생각도 드네요":'포장/배송'}


review
'''가끔 구매해요 가족 모두 만족 감사합니다 또 사러 올게요'''
category_labels
{}


review
'''야식으로 간편하게 먹기 좋아요
만둣국 레시피 1 사골 육수를 냄비에 끓이기 2 만두 넣기 3 5분 정도 지나면 썰어 준 대파를 넣기 4 참기름 김 가루 넣어 주면 완성'''
category_labels
{"야식으로 간편하게 먹기 좋아요":'활용 방법',
"만둣국 레시피 1 사골 육수를 냄비에 끓이기 2 만두 넣기 3 5분 정도 지나면 썰어 준 대파를 넣기 4 참기름 김 가루 넣어 주면 완성":'활용 방법'}

review
'''갈비탕 할인하면 한 두 개씩 사 둬요 냉동이라 더 좋고 대파나 팽이 마늘 넣어 먹으면 더 좋아요
생각보다 실해요 아침에 바쁘고 간단히 차려 먹고 싶을 때 딱 좋네요 집에서 간편하게 해먹기 좋아요
첨가물이 섞여 있지 않은 것도 좋아서 항상 비상식량으로 쟁여둡니다'''
category_labels
{"갈비탕 할인하면 한 두 개씩 사 둬요":'보관',
"냉동이라 더 좋고":'보관',
"대파나 팽이 마늘 넣어 먹으면 더 좋아요":'활용 방법'
"생각보다 실해요":'양',
"아침에 바쁘고 간단히 차려 먹고 싶을 때 딱 좋네요":'활용 방법',
"집에서 간편하게 해먹기 좋아요":'활용 방법',
"첨가물이 섞여 있지 않은 것도 좋아서":'맛/향/풍미',
"항상 비상식량으로 쟁여둡니다":'보관'}


review
'''평소에 갈비탕을 좋아해서 컬리 쇼핑 중 새로 올라온 솔가 원 갈비탕 주문 포장부터 깔끔하더니 국물이 캬 아
큰 대갈비가 4개나 고기도 두툼하고 집에 있는 대파 송송 깍두기에 한 그릇 뚝딱 너무 깔끔하니 맛나네요 4 팩 주문했는데 또 주문해서 쟁겨뒀네요 간편하게 몸 보신 좋네요
가성비 좋습니다 아기도 너무 잘 먹고 어르신들 드시기에도 좋은 것 같아요 대박 나세요'''
category_labels
{"포장부터 깔끔하더니":'포장/배송',
"국물이 캬 아":'맛/향/풍미',
"큰 대갈비가 4개나 고기도 두툼하고":'양',
"너무 깔끔하니 맛나네요":'맛/향/풍미',
"4 팩 주문했는데 또 주문해서 쟁겨뒀네요":'보관',
"간편하게 몸 보신 좋네요":'활용 방법',
"가성비 좋습니다":'가격',
"아기도 너무 잘 먹고":'활용 방법',
"어르신들 드시기에도 좋은 것 같아요":'활용 방법'}


review
'''맛은 그럭저럭인데 포장지는 신경 쓰여야할 것 같습니다 컬리에서 많이 시키는데 해동 후 이렇게 뚫려 내용물이 세어 나온 적은 처음입니다 후기 찾아 보니 저처럼 뚫린 분이 계신 듯합니다'''
category_labels
{"포장지는 신경 쓰여야할 것 같습니다":'포장/배송',
"해동 후 이렇게 뚫려 내용물이 세어 나온 적은 처음입니다":'포장/배송'}


review
'''내용물은 너무 부실해요 국물만 흥건하고 건더기는 별로 없는 \n댓글 알바인가요\n 내용물 거의 아무것도 없잖아요 차라리 청국장 사서 두부 넣고 직접 해드세요 가격 대비 내용물이 부실하다는 느낌'''
category_labels
{"내용물은 너무 부실해요":'양',
"국물만 흥건하고 건더기는 별로 없는":'양',
"내용물 거의 아무것도 없잖아요":'양',
"가격 대비 내용물이 부실하다는 느낌":'가격'}


review
'''내용물 봉지에 이물질인지 벌레인지 묻어 있어서 불쾌합니다 두 개나 굵고 진한 머리카락인가요
국물은 진해 보이지만 찜찜하고 먹기 꺼려져서 버렸어요 위생 상태 별로네요```
category_labels
{"내용물 봉지에 이물질인지 벌레인지 묻어 있어서 불쾌합니다":'포장/배송',
"두 개나 굵고 진한 머리카락인가요":'포장/배송',
"찜찜하고 먹기 꺼려져서 버렸어요 위생 상태 별로네요":'포장/배송'}


review
'''남편 두고 여행을 떠나서 밀키트로 냉장 냉동고에 쌓아 두었어요 넉넉한 고기와 국물이 이거 진짜구나 싶고요 맛은 전문점에서 먹는 것 이상으로 맛있어요 한 팩 먼저 먹고 두 개 더 있었는데 남편도 이게 최고라고 말해 주네요 포장이 잘 되어 있고 조리 과정이랄 것도 없이 끓이기만하면 되니 이 점도 좋아요 후기 쓰고 있는 지금도 먹고 싶어요 팁 파 한두 대 쫑쫑 썰어 밀패 용기에 담아 두어 먹을 때마다 넉넉히 넣어 드세요'''
category_labels
{"밀키트로 냉장 냉동고에 쌓아 두었어요":'보관',
"넉넉한 고기와 국물이 이거 진짜구나 싶고요":'양',
"맛은 전문점에서 먹는 것 이상으로 맛있어요":'맛/향/풍미',
"포장이 잘 되어 있고":'포장/배송',
"조리 과정이랄 것도 없이 끓이기만하면 되니 이 점도 좋아요":'활용 방법',
"파 한두 대 쫑쫑 썰어 밀패 용기에 담아 두어 먹을 때마다 넉넉히 넣어 드세요":'활용 방법'}
"""

In [ ]:
prompt = f"""
카테고리에는 ['맛/향/풍미', '양', '포장/배송', '보관', '가격', '활용 방법'] 항목이 있다.
주어진 review 속 상품에 관한 내용 중 카테고리의 각 항목과 관련되어 있는 일부분인 '파트'를 찾은 후
찾은 '파트'에 카테고리 항목을 태깅한 결과를 dictionary 형태의 category_labels로 출력하라.

'파트'의 단어 개수는 15개가 넘지 않는다.
각 '파트'에는 하나의 카테고리 항목만 태깅한다.
어떠한 항목에도 태깅되는 내용이 없으면 빈 dictionary를 출력한다.

예시:{example}
"""

In [ ]:
def JSON_parsing(text):
    # JSON 형태로 변환
    json_str = text.replace("'", '"')
    json_str = re.sub(r'\\+', '', json_str)
    # JSON 파싱
    return json.loads(json_str)


def tagging(text):
    prompting = (
    f"{prompt}\n"
    "문제:\n"
    "review\n"
    f"'''{text}'''\n"
    "category_labels\n"  
    )
    
    messages = [("system", "You are a consumer analyst extracting meaningful phrases from reviews and categorizing them."),
                ("user", prompting)]
    
    response = llm.invoke(messages)     
    return response
    #return messages

In [ ]:
data=[]

In [ ]:
for i in range(100,num):
    print(i)
    text = df.loc[i, 'cleaned_Kiwi_review']
    response = tagging(text)
    cate_dict = JSON_parsing(response.content)
    row = {'index':i, 'review': text, 'tagging':cate_dict}
    data.append(row)
    
result = pd.DataFrame(data)
result

In [ ]:
result.info()

In [ ]:
result.to_csv("50개씩_카테고리태깅.csv", index=False, encoding="utf-8-sig")

### 태깅된 문장 하나씩으로 이루어진 df 만들기

In [ ]:
df = pd.read_csv('50개씩_감정태깅.csv')
df

In [2]:
import ast

category = pd.read_csv("50개씩_카테고리태깅.csv")
category['tagging'] = category['tagging'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
print(category.loc[0, 'tagging'])

{'하트찜꽁 쫀득한 도가니탕 맛있네요': '맛/향/풍미', '국 간은 싱거운데 소금 간 하면 딱이에요': '맛/향/풍미'}


In [ ]:
df['tagging']=category['tagging']
df

In [ ]:
total = pd.DataFrame(columns=['product_id', 'review_index', 'review', 'category'])
total

In [ ]:
cnt=0

for i in range(df.shape[0]):
    product_id = df.loc[i, 'product_id']
    tagging = df.loc[i, 'tagging']
    
    if i>0 and product_id != df.loc[i-1, 'product_id']:
        cnt=0
    review_index = cnt
    cnt+=1
    
    if len(tagging.keys()) == 0:
        continue
    
    for text in tagging.keys():
        review = text
        category = tagging[text]
        row = [product_id, review_index, review, category]
        total.loc[len(total)] = row

total        

In [ ]:
total['product_id'].nunique()

In [ ]:
total['product_id'].value_counts()

In [ ]:
total.groupby('product_id')['category'].value_counts()

In [ ]:
total.to_csv("카테고리_태깅된_문장.csv", index=False, encoding="utf-8-sig")